# **Bioinformatics Project - Computational Drug Discovery**
# **Part 3**

**Author: Hubert Owusu**

This notebook is dedicated to developing a Generative Adversarial Neural Network (GANN) to generate novel molecular compounds targeting the Replicase polyprotein 1ab of the SARS-CoV-2 virus. The effectiveness of these newly generated compounds to be drug candidates against the Replicase polyprotein 1ab are then evaluated using the regression model from the previous notebook to predict their IC50 values.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('descriptors_output.csv')

In [3]:
df = df.drop(columns='Name')
df.head()

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((908, 881), (228, 881))

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

### **Set Model Parameters**
Define key parameters for the GANN, such as the input dimension (the number of features from the normalized descriptor data).

In [6]:
# Model parameters
input_dim = X_train.shape[1]  # Number of molecular descriptors (features)
latent_dim = 128  # Dimension of the random noise vector for the generator

### **Build the Generator Network**

In [7]:
def build_generator(input_dim, latent_dim):
    model = models.Sequential()

    # Input: random noise (latent vector)
    model.add(layers.Dense(256, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))

    # Hidden layer 1
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))

    # Hidden layer 2
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))

    # Output layer: same dimensionality as the real molecular descriptor data
    model.add(layers.Dense(input_dim, activation='sigmoid'))  # Using 'tanh' to ensure output is between -1 and 1

    return model

In [8]:
generator = build_generator(input_dim, latent_dim)
generator.summary()

C:\Users\huber\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\huber\anaconda3\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 881)            │       903,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,600,113 (6.10 MB)

 Trainable params: 1,596,529 (6.09 MB)

 Non-trainable params: 3,584 (14.00 KB)

### **Build the Discriminator Network**

In [9]:
def build_discriminator(input_dim):
    model = models.Sequential()

    # Input: molecular descriptor data
    model.add(layers.Dense(1024, input_dim=input_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    # Hidden layers
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    # Output: single neuron (real/fake classification)
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

In [10]:
discriminator = build_discriminator(input_dim)
discriminator.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 1024)           │       903,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,559,553 (5.95 MB)

 Trainable params: 1,559,553 (5.95 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Compile the discriminator
discriminator.compile(
    optimizer=tf.keras.optimizers.Adam(0.0002, 0.5),
    loss='binary_crossentropy',
    metrics=['accuracy']
    )

### **Combine the GANN Model**

In [12]:
# Build the GANN
def build_gan(generator, discriminator):
    # Freeze the discriminator's weights when training the generator
    discriminator.trainable = False

    # Build the GAN model
    model = models.Sequential()

    # Add the generator
    model.add(generator)

    # The output of the generator goes into the discriminator
    model.add(discriminator)

    # Compile the combined model (GANN)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.0002, 0.5),
        loss='binary_crossentropy'
        )

    return model

In [13]:
# Rebuild and recompile the discriminator (for individual training)
discriminator = build_discriminator(input_dim)
discriminator.compile(
    optimizer=tf.keras.optimizers.Adam(0.0002, 0.5),
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [14]:
# Build and compile the GANN
gan = build_gan(generator, discriminator)
gan.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 881)            │     1,600,113 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 1)              │     1,559,553 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,159,666 (12.05 MB)

 Trainable params: 1,596,529 (6.09 MB)

 Non-trainable params: 1,563,137 (5.96 MB)

In [15]:
# Compile the GANN
#gan.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='binary_crossentropy')

### **Train the GANN**

In [16]:
# Train the GANN
def train_gan(gan, generator, discriminator, X_train, latent_dim, epochs=800, batch_size=64):
    # Labels for real (1) and fake (0) data
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Train the discriminator on real data
        discriminator.trainable = True  # Ensure the discriminator is trainable
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_data = X_train.iloc[idx]

        # Generate a batch of fake molecular descriptors
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generated_data = generator.predict(noise)
        #generated_data = np.where(generated_data > 0.5, 1, 0) # Binarize the output

        # Train the discriminator (real data as real, generated data as fake)
        d_loss_real = discriminator.train_on_batch(real_data, real_labels)
        d_loss_fake = discriminator.train_on_batch(generated_data, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Freeze the discriminator while training the generator
        discriminator.trainable = False

        # Train the generator (tries to fool the discriminator)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, real_labels)

        # Print progress every 100 epochs
        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}] [G loss: {g_loss}]")
            

In [20]:
train_gan(gan, generator, discriminator, X_train, latent_dim)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step 
0 [D loss: 0.7691621780395508, acc.: 49.21875] [G loss: [array(0.8927411, dtype=float32), array(0.8927411, dtype=float32), array(0.34375, dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
2/2 ━━━━━━━

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━

In [18]:
def generate_descriptors(generator, latent_dim, num_samples=10):
    # Generate random noise as input for the generator
    noise = np.random.normal(0, 1, (num_samples, latent_dim))
    
    # Generate molecular descriptors from the noise
    generated_descriptors = generator.predict(noise)
    
    return generated_descriptors

In [19]:
num_samples = 500  # Number of new descriptors you want to generate
new_descriptors = generate_descriptors(generator, latent_dim, num_samples)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [20]:
new_descriptors = np.where(new_descriptors > 0.5, 1, 0)
generated_df = pd.DataFrame(new_descriptors, columns=X_train.columns)
generated_df.head()

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
generated_df.duplicated().sum()

2

In [22]:
generated_df.shape

(500, 881)

### **Saving The Generator Model**

In [28]:
generator.save('generator_model.keras')

In [17]:
from tensorflow.keras.models import load_model

# Load the entire model
generator = load_model('generator_model.keras')

### Loading The VarianceThreshold model for Feature Selection

In [27]:
import joblib

loaded_selection = joblib.load('variance_threshold_model.pkl')

# Now you can use the loaded model to transform your data
generated_df_cut = loaded_selection.transform(generated_df)

In [28]:
generated_df_cut.shape

(500, 149)

### Loading The RandomForestRegressor Model

In [26]:
import joblib

# Load the saved model
loaded_model = joblib.load('random_forest_model.pkl')


In [29]:
predictions = loaded_model.predict(generated_df_cut)

In [34]:
print("Minimum pIC50 for the generated descriptors = ", predictions.min())
print("Mean pIC50 for the generated descriptors = ", predictions.mean())
print("Maximum pIC50 for the generated descriptors = ", predictions.max())


Minimum pIC50 for the generated descriptors =  5.489001528045253
Mean pIC50 for the generated descriptors =  5.762579402901048
Maximum pIC50 for the generated descriptors =  6.079988986563566


### Comparing the mean IC50 of generated descriptors to the mean IC50 in our original data***

In [35]:
original_data = pd.read_csv("Replicase_polyprotein_1ab_05_bioactivity_data_3class_pIC50_pubchem_fp.csv")

In [37]:
df_y = original_data['pIC50']

In [38]:
df_y.describe()

count    1136.000000
mean        5.992470
std         1.168985
min         3.371611
25%         5.118616
50%         6.073151
75%         6.744727
max        10.886057
Name: pIC50, dtype: float64

As we can see, our GANN model is performing very well given that that it is producing molecular descriptors with a pIC50 around the mean pIC50 in our real world data. This is in fact very fascinating. 

### **Testing To See If There Are Any Generated Descriptors In Our Real Dataset.**

In [39]:
# Reset indices if they are not aligned
generated_df.reset_index(drop=True, inplace=True)
df_train.reset_index(drop=True, inplace=True)

# Check for duplicate rows
duplicates = generated_df[generated_df.apply(tuple, 1).isin(df_train.apply(tuple, 1))]

# Print duplicates
if not duplicates.empty:
    print("Found duplicates:")
    print(duplicates)
else:
    print("No duplicates found.")

No duplicates found.


None of the descriptors generated with our GANN generator can be found in our real world dataset. Meaning these are all "novel" molecules, or already existing drug candidates that could be repurposed.  

## Converting the pIC50 to IC50 in Molar units

In [41]:
y_pred_df = pd.DataFrame(predictions, columns=["pIC50"])

In [43]:
def reverse_pIC50(input, term):
    standard_value_norm = []

    for pIC50_value in input[term]:
        molar = 10**(-pIC50_value)  # Converts pIC50 back to Molar
        nM_value = molar * 10**9    # Converts Molar back to nM (nanomolar)
        standard_value_norm.append(nM_value)

    input['standard_value_norm'] = standard_value_norm
    x = input.drop(columns='pIC50')

    return x

In [44]:
norm_pred_IC50 = reverse_pIC50(y_pred_df, 'pIC50')

In [55]:
print(f"Minimum IC50 for the generated descriptors = {norm_pred_IC50['standard_value_norm'].min()} M")
print(f"Mean IC50 for the generated descriptors = {norm_pred_IC50['standard_value_norm'].mean()} M")
print(f"Maximum IC50 for the generated descriptors = {norm_pred_IC50['standard_value_norm'].max()} M")

Minimum IC50 for the generated descriptors = 831.7848643791389 M
Mean IC50 for the generated descriptors = 1772.9016168606622 M
Maximum IC50 for the generated descriptors = 3243.3847617726124 M
